# Python Logging

Printing text to screen, in a running program, usually has two purposes (ignoring CLI interface):

### Debug statements
Programmers will output values of variables and steps of a program to help them debug errors. This allows them to view the internal state of a program as it runs. As we saw in the _debugger_ notebook, breakpoints are usually more effective in finding bug.

### Logging statements
There are times when you want to record parts of the internal state of a program for posterity. If the program performs in an errant manner (perhaps due to a bug, perhaps due to an error in the environment where the program was run), you want a record of it. 

In [1]:
i_val = 5
j_val = 5

total = 0

for i in range(i_val):
    for j in range(j_val):
        total += i + j
        print("Current values of i, j and total:", i, j, total) # <= This is TOO much detail, don't leave this in the final program

if total > 100: print("ERROR: Values must never exceed 100")
else: print("Program executed normally and the total value is", total)

Current values of i, j and total: 0 0 0
Current values of i, j and total: 0 1 1
Current values of i, j and total: 0 2 3
Current values of i, j and total: 0 3 6
Current values of i, j and total: 0 4 10
Current values of i, j and total: 1 0 11
Current values of i, j and total: 1 1 13
Current values of i, j and total: 1 2 16
Current values of i, j and total: 1 3 20
Current values of i, j and total: 1 4 25
Current values of i, j and total: 2 0 27
Current values of i, j and total: 2 1 30
Current values of i, j and total: 2 2 34
Current values of i, j and total: 2 3 39
Current values of i, j and total: 2 4 45
Current values of i, j and total: 3 0 48
Current values of i, j and total: 3 1 52
Current values of i, j and total: 3 2 57
Current values of i, j and total: 3 3 63
Current values of i, j and total: 3 4 70
Current values of i, j and total: 4 0 74
Current values of i, j and total: 4 1 79
Current values of i, j and total: 4 2 85
Current values of i, j and total: 4 3 92
Current values of i,

## Python's built-in logging library

In [4]:
import logging

In [5]:
i_val = 5
j_val = 6

total = 0

for i in range(i_val):
    for j in range(j_val):
        total += i + j
        logging.debug("Current values of i, j and total:", i, j, total) # <= This is TOO much detail, don't leave this in the final program

if total > 100: logging.error("ERROR: Values must never exceed 100")
else: logging.info("Program executed normally and the total value is", total)

logging.info("Program complete")

ERROR:root:ERROR: Values must never exceed 100


Python logging library is a very powerful utility, based on Java's _log4j_ library. Logging is often one of the ways professional code is different from student code.

Note that only the `error` message is printed. Python's logging library provides 5 (relevant) logging levels, in order of increasing importance:
* `logging.debug()`
* `logging.info()`
* `logging.warning()` <= Python's default logging level (this, `error` and `critical` are enabled by default)
* `logging.error()`
* `logging.critical()`

The names are self-explanatory. Instead of writing message to the console with a single `print()` statement, you can now segregate them by priority. Under normal conditions, `warning`, `error` and `critical` messages are printed. You can think of their importance as:

* `warning`: Something unexpected has happened, perhaps an unexpected if/else clause was executed or perhaps an unexpected input value was received. This may or may not be a big deal.
* `error`: Something wrong has happened! 
* `critical`: This program is about to go bye bye. 

Note that `critical` is not always used. Often the distinction between `error` and `critical` is not necessary.

Message levels `debug` and `info` are not printed by default. Note that _log4j_'s default was `info`, not warning. In other words, all messages of priority `info` and higher were printed. Python increases the default to `warning`. I do not agree with this decision.

Here is how these two levels are used:
* `debug`: Debug messages generally display values of variables we already know. We don't need to print them. Howver, if a program start acting up, debug logs can be enabled to peek into programs more closely.
* `info`: This is the most common level used by programmers. This is what you want to see as your program is running. This helps to confirm that your program is making progress, responding to client requests, etc.

The `log4j` library provides a priority lower than `debug`, called `trace`. You use `trace` to log far more detail than you would ever need. I tis generally not used, unless the programmer is losign his or her mind trying to track down errors.

#### Please note that for the remainder of this lecture, we will assume that code is being run in a .py file, on the command line. 
Jupyter interferes with Python's logging (there are work=arounds, but add more complexity than we need at this point in the lecture)

You can change the default logging level:

In [2]:
%%writefile python_logging_01.py

import logging

logging.basicConfig(level=logging.INFO)


def maximum(numbers):
    #pass # pass means "do nothing", add your code here
    max_value = 0
    for num in numbers:
        logging.debug(f"num:{num}, max_value:{max_value}")
        if num > max_value:
            logging.debug("max > max_value branch taken. Setting new max_value")
            max_value = num
    return max_value

def main():
    logging.info("Let us find the maximum value from the following list:")
    list_of_nums = [1,2,3]
    max_value = maximum(list_of_nums)
    logging.info(max_value)

    logging.info("Let us find the maximum value from another list:")
    list_of_nums = [-1, -2, -3]
    max_value = maximum(list_of_nums)
    logging.info(max_value)
    
if __name__ == "__main__":
    logging.warning(f"This program is being run from the command line")
    main()


Overwriting python_logging_01.py


## Why can't you just do `print("INFO: answering client query XYZ")`?

#### Admins can control logging level after the program is deployed

In the code above, we simply imported the logging library and started using it. This is perfectly fine for small programs or programs which run on an enviornmet you control. However, deploying in a more restricted enviornment is very different. What if you program is running on a tightly controlled server or perhaps even on a client's computer. Do you ask your client to edit yoru code and change the default logging level Clearly not!

Source: https://gist.github.com/panamantis/5797dda98b1fa6fab2f739a7aacc5e9d

In [6]:
%%writefile logging.json
{
    "version": 1,
    "disable_existing_loggers": false,

    "handlers": {
        "console": {
            "class": "logging.StreamHandler",
            "level": "DEBUG",
            "stream": "ext://sys.stdout"
        },
        "file": {
            "class": "logging.handlers.TimedRotatingFileHandler",
            "level": "DEBUG",
            "when": "D",
            "backupCount": 0,
            "filename": "./logs/training-stats.log"
        }
    },
    "loggers": {
        "root": {
            "level": "DEBUG",
            "handlers": ["console"]
        },
        "app": {
            "level": "DEBUG",
            "handlers": ["file"],
            "propagate": true,
            "qualname": "app"
        }
    }
}

Overwriting logging.json


In [ ]:
%%writefile python_logging_02.py

import logging.config
import json

with open("logging.json", "r") as f:
        json_config = json.load(f)
        logging.config.dictConfig(json_config)

def maximum(numbers):
    #pass # pass means "do nothing", add your code here
    max_value = 0
    for num in numbers:
        logging.debug(f"num:{num}, max_value:{max_value}")
        if num > max_value:
            logging.debug("max > max_value branch taken. Setting new max_value")
            max_value = num
    return max_value

def main():
    logging.info("Let us find the maximum value from the following list:")
    list_of_nums = [1,2,3]
    max_value = maximum(list_of_nums)
    logging.info(max_value)

    logging.info("Let us find the maximum value from another list:")
    list_of_nums = [-1, -2, -3]
    max_value = maximum(list_of_nums)
    logging.info(max_value)
    
if __name__ == "__main__":
    logging.warning(f"This program is being run from the command line")
    main()


```
This program is being run from the command line
Let us find the maximum value from the following list:
num:1, max_value:0
max > max_value branch taken. Setting new max_value
num:2, max_value:1
max > max_value branch taken. Setting new max_value
num:3, max_value:2
max > max_value branch taken. Setting new max_value
3
Let us find the maximum value from another list:
num:-1, max_value:0
num:-2, max_value:0
num:-3, max_value:0
0
```

Notice that system admins can now control the logging of this application, without ever touching the code!

#### The _format_ of the logs can be changed
While writing code, programmers may not be very dilligent about recording information which may not be relevant to the problem they are currently solving. Things like timestamps are very important -- necesary even. The _logging_ module lets you control this

_logging.json_

Notice the `formatters` section as well sa the `"formatter": "simple",` entry in the `handlers` section
```json
{
    "version": 1,
    "formatters": {
        "simple": {
            "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
        }
    },
    "handlers": {
        "console": {
            "class": "logging.StreamHandler",
            "level": "DEBUG",
            "formatter": "simple",
            "stream": "ext://sys.stdout"
        }
    },
    "loggers": {
        "root": {
            "level": "DEBUG",
            "handlers": ["console"]
        }
    }
}
```

We are adding additional information:
* `asctime`: Timestamp
* `name`: Name of the logger
* `levelname`: _debug_, _info_, etc.
* `message`: Actual text

In [7]:
%%writefile logging_fancy.json
{
    "version": 1,
    "disable_existing_loggers": false,
    "formatters": {
        "simple": {
            "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
        },
        "verbose": {
            "format": "%(asctime)s - %(name)s - %(levelname)s - %(funcName)s:%(lineno)d - %(message)s"
        }
    },
    "handlers": {
        "console": {
            "class": "logging.StreamHandler",
            "level": "DEBUG",
            "formatter": "simple",
            "stream": "ext://sys.stdout"
        },
        "file": {
            "class": "logging.handlers.TimedRotatingFileHandler",
            "level": "DEBUG",
            "formatter": "verbose",
            "when": "D",
            "backupCount": 0,
            "filename": "./logs/training-stats.log"
        },
        "uvicorn": {
            "class": "logging.handlers.TimedRotatingFileHandler",
            "level": "DEBUG",
            "formatter": "verbose",
            "when": "D",
            "backupCount": 0,
            "filename": "./logs/uvicorn.log"
        }
    },
    "loggers": {
        "root": {
            "level": "DEBUG",
            "handlers": ["console"]
        },
        "app": {
            "level": "DEBUG",
            "handlers": ["file"],
            "propagate": true,
            "qualname": "app"
        },
        "uvicorn": {
            "level": "DEBUG",
            "handlers": ["uvicorn"],
            "propagate": true,
            "qualname": "uvicorn"
        }
    }
}

Overwriting logging_fancy.json


WIthout changing a single line in the code, the output now looks like this:
```
2024-03-01 21:19:03,386 - root - WARNING - This program is being run from the command line
2024-03-01 21:19:03,386 - root - INFO - Let us find the maximum value from the following list:
2024-03-01 21:19:03,386 - root - DEBUG - num:1, max_value:0
2024-03-01 21:19:03,386 - root - DEBUG - max > max_value branch taken. Setting new max_value
2024-03-01 21:19:03,386 - root - DEBUG - num:2, max_value:1
2024-03-01 21:19:03,386 - root - DEBUG - max > max_value branch taken. Setting new max_value
2024-03-01 21:19:03,386 - root - DEBUG - num:3, max_value:2
2024-03-01 21:19:03,386 - root - DEBUG - max > max_value branch taken. Setting new max_value
2024-03-01 21:19:03,394 - root - INFO - 3
2024-03-01 21:19:03,394 - root - INFO - Let us find the maximum value from another list:
2024-03-01 21:19:03,394 - root - DEBUG - num:-1, max_value:0
2024-03-01 21:19:03,394 - root - DEBUG - num:-2, max_value:0
2024-03-01 21:19:03,394 - root - DEBUG - num:-3, max_value:0
2024-03-01 21:19:03,394 - root - INFO - 0
```

#### File or module specific logger can provide even more useful information
As the current output shows, there is no way for us to know _which_ file or library produced these logs. If you program consists of multiple libraries, it is useful to make the logger name more specific

In [ ]:
%%writefile python_logging_03.py

import logging.config
import json
import os

with open("logging_fancy.json", "r") as f:
        json_config = json.load(f)
        logging.config.dictConfig(json_config)

fname = os.path.basename(__file__)
log = logging.getLogger(fname) # <= This lines makes the logger name more useful

def maximum(numbers):
    #pass # pass means "do nothing", add your code here
    max_value = 0
    for num in numbers:
        log.debug(f"num:{num}, max_value:{max_value}")
        if num > max_value:
            log.debug("max > max_value branch taken. Setting new max_value")
            max_value = num
    return max_value

def main():
    log.info("Let us find the maximum value from the following list:")
    list_of_nums = [1,2,3]
    max_value = maximum(list_of_nums)
    log.info(max_value)

    log.info("Let us find the maximum value from another list:")
    list_of_nums = [-1, -2, -3]
    max_value = maximum(list_of_nums)
    log.info(max_value)
    
if __name__ == "__main__":
    log.warning(f"This program is being run from the command line")
    main()


Output:
```
2024-03-01 21:34:27,510 - maximum_bad_logging.py - WARNING - This program is being run from the command line
2024-03-01 21:34:27,510 - maximum_bad_logging.py - INFO - Let us find the maximum value from the following list:
2024-03-01 21:34:27,510 - maximum_bad_logging.py - DEBUG - num:1, max_value:0
2024-03-01 21:34:27,510 - maximum_bad_logging.py - DEBUG - max > max_value branch taken. Setting new max_value
2024-03-01 21:34:27,510 - maximum_bad_logging.py - DEBUG - num:2, max_value:1
2024-03-01 21:34:27,510 - maximum_bad_logging.py - DEBUG - max > max_value branch taken. Setting new max_value
2024-03-01 21:34:27,510 - maximum_bad_logging.py - DEBUG - num:3, max_value:2
2024-03-01 21:34:27,510 - maximum_bad_logging.py - DEBUG - max > max_value branch taken. Setting new max_value
2024-03-01 21:34:27,510 - maximum_bad_logging.py - INFO - 3
2024-03-01 21:34:27,510 - maximum_bad_logging.py - INFO - Let us find the maximum value from another list:
2024-03-01 21:34:27,510 - maximum_bad_logging.py - DEBUG - num:-1, max_value:0
2024-03-01 21:34:27,510 - maximum_bad_logging.py - DEBUG - num:-2, max_value:0
2024-03-01 21:34:27,510 - maximum_bad_logging.py - DEBUG - num:-3, max_value:0
2024-03-01 21:34:27,510 - maximum_bad_logging.py - INFO - 0
```

#### These logs are just being written to the console, do we have to redirect output to a file?
If a program is writing output to a console, most command line shells will let you redirect the output to a file `python maximum_bad_logging.py > mbl.log`. 

The _logging_ library provides a better solution

_logging.json_

Notice that we have added a new `handler` called `file`. The `loggers.root.handlers` section now also lists `"file"` in the list
```
{
    "version": 1,
    "formatters": {
        "simple": {
            "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
        }
    },
    "handlers": {
        "console": {
            "class": "logging.StreamHandler",
            "level": "DEBUG",
            "formatter": "simple",
            "stream": "ext://sys.stdout"
        },
        "file": {
            "class": "logging.handlers.TimedRotatingFileHandler",
            "level": "DEBUG",
            "formatter": "simple",
            "when": "D",
            "backupCount": 0,
            "filename": "./maximum_bad_logging.log"
        }
    },
    "loggers": {
        "root": {
            "level": "DEBUG",
            "handlers": ["console", "file"]
        }
    }
}
```

The console output is exactly the same, but now there is also a file containing the logs. Note that we are using the `TimedRotatingFileHandler` above, which will roll over the log every day and _automatically_ protect your logs! This is VERY useful for long running processes, such as application servers.

Here are interesting _FileHandlers_ in the _logging_ library. Note that they are presented here only to show how powerful (and over the top) logging can get. Students are encouraged to keep things simple:

* `NullHandler`: Doesn't do anything
* `WatchedFileHandler`: " ...watches the file it is logging to. If the file changes, it is closed and reopened using the file name. A file change can happen because of usage of programs such as newsyslog and logrotate which perform log file rotation."
* `RotatingFileHandler`: " supports rotation of disk log files."
* `TimedRotatingFileHandler`: "supports rotation of disk log files at certain timed intervals."
* `SocketHandler`: "sends logging output to a network socket."
* `SysLogHandler`: " supports sending logging messages to a remote or local Unix syslog."
* `NTEventLogHandler`: "supports sending logging messages to a local Windows NT, Windows 2000 or Windows XP event log"
* `SMTPHandler`: " supports sending logging messages to an email address via SMTP."
* `MemoryHandler`: " supports buffering of logging records in memory, periodically flushing them to a target handler. Flushing occurs whenever the buffer is full, or when an event of a certain severity or greater is seen."
* `HTTPHandler`: " supports sending logging messages to a web server, using either GET or POST semantics."
* `QueueHandler`: "supports sending logging messages to a queue, such as those implemented in the queue or multiprocessing modules"

(quoted text copied directly from Python docs)

#### Logging can be very expensive! Don't execute discarded logs
A pattern common in Java (remember that _logging_ is inspired by log4java), but not seen as much in Python is to wrap uncommon logging statements in an `if` clause:

```python
log.debug(f"The value of total is {complicatedTotalCalculation()}")
log.info("Process complete")
```

Let's say that the default logging level is set to `info`, which means that the `debug` statement will not be logged.

The `debug` statement will STILL BE EXECUTED! It will just get filtered out before it is logged to console or file! Particularly the `complicatedTotalCalculation` function will be executed, even if its value is thrown away!

Java developers use this pattern:

```python
if log.getEffectiveLevel() == logging.DEBUG: log.debug(f"The value of total is {complicatedTotalCalculation()}")
if log.getEffectiveLevel() == logging.INFO: log.info("Process complete")
```

Notice how this only executes the log statement if that level of log is actually enabled. It does make the code look terribe but if your program is littered with `debug` statements, when the default level is set to `info`, such optimizations may be useful.

#### Logging can be very expensive! Look into async logs
Most logging statements are blocking (see asyncio section for more details). Python now provides facilities to log in a non-blocking manner. See the docs for `QueueHandler`. A nice overview is provided in this blog: https://medium.com/@augustomen/using-logging-asynchronously-c8e854de874c 